In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits import mplot3d
from matplotlib.animation import FuncAnimation
%matplotlib qt

# PSO Alg - origional

In [9]:
def F(X):
    trm1 = -20 * np.exp(-0.2 * np.sqrt(np.sum(X**2, axis=0) / d))
    trm2 = -np.exp(np.sum(np.cos(2 * np.pi * X), axis=0) / d)
    return trm1 + trm2 + 20 + np.e

# Set up parameters 

# dimension 
d = 2
# boundary of velocity search domain
Vmax = 4
# boundary of space search domian 
Xmin = -4
Xmax = 4

# Number of particles 
Number_particles = 20
# Personal coefficient c1, social coeff c2, we keep equal
c1 = c2 = 2
# Maximum iterations
MaxIts = 1000
# Convergence 
convergence = 0.001


# Set up initialization 
X = np.random.uniform(Xmin, Xmax, (d, Number_particles))
V = np.random.uniform(-Vmax, Vmax, (d, Number_particles))


# This is defined as in the paper

# As per definiton for local best at it = 0 

local_best = X
# this should be a vector of length d 
local_best_objective = F(X)

# particle which minimises F(X) most 
# This gives the index
argmin_ = local_best_objective.argmin()
# This should be the d vector of xi
global_best = local_best[:, argmin_]

# initially this is just defined as the min of 
# local_best_objective 
global_best_objective = local_best_objective.min()


def R_generator(d):
    '''
    d-dim diagonal matrices with random
    numbers uniformly distributed in [0, 1]
    '''
    x1 = np.random.rand(d)
    x2 = np.random.rand(d)
    R_1 = np.diag(x1)
    R_2 = np.diag(x2)
    return R_1, R_2

current_it = 0

while current_it < MaxIts:

    R_1, R_2 = R_generator(d)

    # Cheak that y_global_best.reshape(-1, 1) is a matrix of the 
    # same columns and not the rest of the matrix is zero
    # because X at this point is still a matrix so has to match the dim
    
    # the assumption here (global_best.reshape(-1, 1)- X)
    # Is that you are taking every particle away from the global best 
    # So it is like a matrix of the same particle

    V = V + (c1 * R_1) @ (local_best- X) + (c2 * R_2) @ (global_best.reshape(-1, 1)- X)
    X = X + V
    obj_val = F(X)

    # Domain - constraints 
    # For space
    X = np.clip(X, Xmin, Xmax)
    # For velocity
    V = np.clip(V, -Vmax, Vmax)

        # Local best - 
    # we need to update each particle so that 
    # If the objective value for the partice is less for X vs Pbest
    Bool = obj_val <= local_best_objective
    
    # For the true update - otherwise keep particle
    # Recall - local best is still the matrix of particles 
    local_best[:, Bool] = X[:, Bool]

    # This is the local best - obj just feed the matrix through
    local_best_objective = F(local_best)
    
    # Cheak if the minimum value of local best obj < min of global of previous
    # If it is then we know that the new global obj is this value
    # Also the value of this particle is given by selecting 
    # the particle with this argument
    
    # Otherwise global_best remains the same
    if local_best_objective.min() < global_best_objective:
        global_best_objective = local_best_objective.min()
        global_best = local_best[:, local_best_objective.argmin()]


    # Convergence cheak 
    if abs(global_best_objective) < convergence:
        print("Covergence criteria have been met after "+ str(current_it)+" iterations.")
        break 
    current_it += 1


print("PSO f({}) = {}".format(global_best, global_best_objective))

# abs(global best - actual global best) < conv

PSO f([ 0.01654525 -0.02382574]) = 0.10432184975049852


# Plot for Quiver - 

In [11]:
# Define function 
def f(X):
    x1 , x2 = X
    return x1**2 + x2**2

d = 2

def F(X):
    '''
    axis = 0 : means over the rows in this case
    '''
    trm1 = -20 * np.exp(-0.2 * np.sqrt(np.sum(X**2, axis=0) / d))
    trm2 = -np.exp(np.sum(np.cos(2 * np.pi * X), axis=0) / d)
    return trm1 + trm2 + 20 + np.e

# Set up parameters 
d = 2
Vmax = 4
Xmin = -4
Xmax = 4
Number_particles = 20
c1 = c2 = 0.1
MaxIts = 1000
convergence = 0.001

# Set up initialization 
X = np.random.uniform(Xmin, Xmax, (d, Number_particles))
V = np.random.uniform(-Vmax, Vmax, (d, Number_particles))

# As per definition for local best at it = 0 
local_best = X
local_best_objective = F(X)

# particle which minimizes F(X) most 
argmin_ = local_best_objective.argmin()
global_best = local_best[:, argmin_]
global_best_objective = local_best_objective.min()

def R_generator(d):
    '''
    d-dim diagonal matrices with random
    numbers uniformly distributed in [0, 1]
    '''
    x1 = np.random.rand(d)
    x2 = np.random.rand(d)
    R_1 = np.diag(x1)
    R_2 = np.diag(x2)
    return R_1, R_2

x1 = np.linspace(-4, 4, 100)
x2 = np.linspace(-4, 4, 100)
X1, X2 = np.meshgrid(x1, x2)
Z = F(np.array([X1, X2]))

fig, ax = plt.subplots()

def init():
    # We don't really need an initialisation here 
    # but keep it incase 
    ax.imshow(Z, extent=[-4, 4, -4, 4], origin='lower', cmap='magma', alpha=0.5)
    ax.contourf(X1, X2, Z, 10, cmap='magma')
    return ax,

def update(frame):
    global X, V, local_best, local_best_objective, global_best, global_best_objective
    # Global is just what is being updated each frame

    # Plot
    
    # So X1, X2 - creates the meshgrid - evaluated Z - defines the colors 
    # 10 is how detailed / how many contours 
    ax.imshow(Z, extent=[-4, 4, -4, 4], origin='lower', cmap='magma', alpha=0.5)
    ax.contourf(X1, X2, Z, 10, cmap='magma')
    ax.set_title('Particle Swarm Optimization')
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')

    
    # update rule
    R_1, R_2 = R_generator(d)
    V = 0.2*V + (c1 * R_1) @ (local_best- X) + (c2 * R_2) @ (global_best.reshape(-1, 1)- X)
    X = X + V
    obj_val = F(X)

    # Domain - constraints 
    X = np.clip(X, Xmin, Xmax) # space
    V = np.clip(V, -Vmax, Vmax) # velocity
    
        # update global and local best
    Bool = obj_val <= local_best_objective
    local_best[:, Bool] = X[:, Bool]
    local_best_objective = F(local_best)

    if local_best_objective.min() < global_best_objective:
        global_best_objective = local_best_objective.min()
        global_best = local_best[:, local_best_objective.argmin()]
    

    # plot X
    ax.scatter(X[0], X[1], c='w')
    # Plot the global best particle each iteration
    ax.scatter(global_best[0], global_best[1], color='w', marker='x', label='Global best', s=100)
    
    # This will give an arrow to each particle 
    # we go through all the particles for the iteration
    # The arrow depends on the velocity and space
    for i in range(Number_particles):
        ax.quiver(X[0, i], X[1, i], V[0, i], V[1, i], color='w', width=0.005, angles='xy', scale_units='xy', scale=1)
        
    # This returns the axis object - so we can update plot
    return ax,


ani = FuncAnimation(fig, update, frames=MaxIts, init_func=init, blit=False)
plt.show()

